In [52]:
import sys

import sys
sys.path.append('wu_server/')

In [53]:
from GenerateLabelCPM import *
from modelCPM import *

In [54]:
sym = mxnetModule()

In [55]:
cmodel = intializeModel(sym)

self._data_shapes:  [DataDesc[data,(1, 3, 368, 368),<type 'numpy.float32'>,NCHW]]
self._label_shapes:  [DataDesc[heatmaplabel,(1, 19, 46, 46),<type 'numpy.float32'>,NCHW], DataDesc[partaffinityglabel,(1, 38, 46, 46),<type 'numpy.float32'>,NCHW]]


In [46]:
cocodata = cocoIter('pose_io/data.json',
                    'data', (1,3, 368,368),
                    ['heatmaplabel','partaffinityglabel'],
                    [(1, 19, 46, 46),(1,38,46,46)])

In [47]:
import time
startt = time.time()

cmodel.init_optimizer(optimizer='sgd', 
                      optimizer_params=(('learning_rate', 0.01), ))

# train one epoch, i.e. going over the data iter one pass
for batch in cocodata:
    cmodel.forward(batch, is_train=True)       # compute predictions
    prediction=cmodel.get_outputs()
    #print prediction
    #print prediction[11].asnumpy().reshape(-1).shape
    #print batch.label[0].asnumpy().reshape(-1).shape
    print '-------------'
    print np.mean((prediction[11].asnumpy().reshape(-1)-batch.label[0].asnumpy().reshape(-1))**2)
    print np.mean((prediction[10].asnumpy().reshape(-1)-batch.label[1].asnumpy().reshape(-1))**2)
    cmodel.backward()                          # compute gradients
    cmodel.update()                            # update parameters using SGD
    
endt = time.time()
print endt-startt

-------------
0.00127089
0.00197374
-------------
0.00140862
0.00445809
-------------
0.00114914
0.00167099
-------------
0.0010632
0.0015162
117.425956011


### Fake Data

In [48]:
cocoIterFake = cocoIter('pose_io/data.json',
                    'data', (1,3, 368,368),
                    ['heatmaplabel','partaffinityglabel'],
                    [(1, 19, 46, 46),(1,38,46,46)])

In [49]:
import time
startt = time.time()

cmodel.init_optimizer(optimizer='sgd', 
                      optimizer_params=(('learning_rate', 0.01), ))

# train one epoch, i.e. going over the data iter one pass
for batch in cocoIterFake:
    cmodel.forward(batch, is_train=True)       # compute predictions
    prediction=cmodel.get_outputs()
    #print prediction
    #print prediction[11].asnumpy().reshape(-1).shape
    #print batch.label[0].asnumpy().reshape(-1).shape
    print '-------------heatmap-------------'
    print np.mean((prediction[11].asnumpy().reshape(-1)-batch.label[0].asnumpy().reshape(-1))**2)
    print '-------------part affinity graph-------------'
    print np.mean((prediction[10].asnumpy().reshape(-1)-batch.label[1].asnumpy().reshape(-1))**2)
    cmodel.backward()                          # compute gradients
    cmodel.update()                            # update parameters using SGD
    
endt = time.time()
print endt-startt

-------------heatmap-------------
0.00146501
-------------part affinity graph-------------
0.00225034
-------------heatmap-------------
0.00195787
-------------part affinity graph-------------
0.00768836
-------------heatmap-------------
0.000803418
-------------part affinity graph-------------
0.000458169
-------------heatmap-------------
0.000942113
-------------part affinity graph-------------
0.00121227
139.15998888


In [50]:
### Comparsion

In [51]:
import matplotlib.pylab as plt
%matplotlib inline

dataitera = cocodata.next()


newsimage = dataitera.data[0].asnumpy()[0]

newsimage.shape

heatmap1 = dataitera.label[0].asnumpy()[0]

pagmap1 = dataitera.label[1].asnumpy()[0]

heatmap1.shape

pagmap1.shape

#oriImg[:,:,[2,1,0]]
f = plt.imshow(newsimage[0,:,:]) # reorder it before displaying

plt.figure();  # plt.axis('off')

for i in range(18,19):
    plt.imshow(cv.resize(heatmap1[i,:,:], (368, 368)))

plt.imshow(newsimage[0,:,:], alpha=0.5)

plt.show()

cmodel.forward(dataitera)       # compute predictions
prediction=cmodel.get_outputs()

heatmap2 = prediction[11].asnumpy()[0]

plt.figure();  # plt.axis('off')

for i in range(18,19):
    plt.imshow(cv.resize(heatmap2[i,:,:], (368, 368)), alpha=(1 - 1.0 * i / 10))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()

differentmap = heatmap2[18,:,:]-heatmap1[18,:,:]

differentmap

differentmap.max()

plt.figure();  # plt.axis('off')

for i in range(7,8):
    plt.imshow(cv.resize(differentmap, (368, 368)), alpha=(1 - 1.0 * i / 10))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()

StopIteration: 